# 导入模块

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.rcParams["font.sans-serif"]=["SimHei"]
plt.rcParams["axes.unicode_minus"]=False

import warnings
warnings.filterwarnings("ignore")
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import pymysql
from sqlalchemy import create_engine, column
import json
# from jsonpath_ng import jsonpath, parse
from jsonpath import jsonpath
import datetime
from datetime import timedelta ,time
import re
import swifter
import os
import sys
# 获取当前工作目录
current_dir = os.getcwd()
# 将模块所在的目录添加到 sys.path
module_dir = os.path.join(current_dir, '../Class_Model')
sys.path.append(module_dir)
from All_Class import All_Model, Week_Model, Data_Clean, Risk_Data

In [2]:
pd.set_option('display.max_columns', None)
# 实例化All_Model类
all_models = All_Model()
week_models = Week_Model()
clean = Data_Clean()
risk_data = Risk_Data()

# 连接数据库

In [3]:
def query(sql,
          host="rr-wz9wx0w3yti9d4f6wro.mysql.rds.aliyuncs.com",
          user="ylc",
          password="1O8t5lcJ5aMhwwPEUUjS",
          database = '' ,
          port=3306
         ):   
    conn = pymysql.connect(
        host=host,
        user=user,
        port=port,
        password=password,
        max_allowed_packet=1073741824,
        charset="utf8")
    try:
        df = pd.read_sql(sql, con=conn)
        conn.close()
    except:
        print('error')
        conn.close()
        raise
    return df

# 导入数据

In [4]:
date1 = '2023-11'
date2 = '2023-11'
sql1=f'''-- 风控-百融 
SELECT rb.trace_id, rb.query_rs
from db_credit.risk_bairong rb 
where date_format(rb.query_time, '%Y-%m') >= '{date1}'
and date_format(rb.query_time, '%Y-%m') <= '{date2}'
;
'''

br_df =query(sql1)
br_df.shape

(15419, 2)

In [5]:
product_list = ['RuleApplyLoan_cashon', 'Rule_D_ApplyLoanC', 'Rule_D_ApplyLoanUsury', 'ApplyLoan_d', 'ApplyLoanMon', 'ApplyLoanStr', 'ApplyLoanC', 'ApplyLoanUsury']
# 数据类型转换为category（分类）类型
br_df = br_df.astype({'query_rs': 'category', 'trace_id': 'category'})

# 获取产品的最大权重

In [6]:

# # 获取对应的产品名称是否调用
# def get_num(query_rs):
#     for name in product_list:
#         if name.lower() in query_rs:
#             return jsonpath(json.loads(query_rs), f'$..Flag.{name.lower()}')[0]
# br_df.loc[:, 'num'] = br_df.swifter.apply(lambda x: get_num(x.query_rs), axis=1)
# br_df.loc[:, 'num'] = br_df.num.astype('int8')
# # 获取已经调用的产品
# br_df_new = br_df[br_df.num==1]
# # 获取权重
# def check(query_rs, num, key=None):
#     for name in product_list:
#         if name.lower() in query_rs and num==1 and key=='final_weight':
#             return jsonpath(json.loads(query_rs), '$..final_weight')[0]
# br_df_new.loc[:, 'final_weight'] = br_df_new.swifter.apply(lambda x: check(x.query_rs, x.num, 'final_weight'), axis=1)
# br_df_new[['trace_id', 'final_weight']].to_excel('F:/需求/总需求/权重分布1.xlsx')

# 获取每个版本命中的数据

In [7]:
# # 获取命中了哪个产品，并修改字段类型
# # 预先解析所有的 JSON 字符串到一个新的列中
# br_df['parsed_query_rs'] = br_df['query_rs'].astype(str).apply(json.loads)
# 
# # 创建一个字典来缓存 jsonpath 的编译表达式
# jsonpath_exprs = {name: parse(f'$..Flag.{name.lower()}') for name in product_list}
# 
# # 定义一个辅助函数来提取指定名称的值
# def extract_value(parsed_json, expr):
#     match = expr.find(parsed_json)
#     return match[0].value if match else 0
# 
# # 使用 apply 和 swifter 来加速
# for name in product_list:
#     expr = jsonpath_exprs[name]
#     br_df[f'{name}_num'] = br_df['parsed_query_rs'].swifter.apply(lambda x: extract_value(x, expr))

In [9]:
# 获取命中了哪个产品，并修改字段类型
for name in product_list:
    br_df.loc[:, f'{name}_num'] = br_df.swifter.apply(lambda x: jsonpath(json.loads(x.query_rs), f'$..Flag.{name.lower()}')[0] if name.lower() in x.query_rs else 0, axis=1)

br_df = br_df.astype({'RuleApplyLoan_cashon_num': 'int8', 'Rule_D_ApplyLoanC_num': 'int8', 'Rule_D_ApplyLoanUsury_num': 'int8', 'ApplyLoan_d_num': 'int8', 'ApplyLoanMon_num': 'int8', 'ApplyLoanStr_num': 'int8', 'ApplyLoanC_num': 'int8', 'ApplyLoanUsury_num': 'int8'})

# 获取信贷意向验证 线上现金分期（产品代号：RuleApplyLoan_cashon）
br_df_cae = br_df[br_df.RuleApplyLoan_cashon_num==1][['trace_id', 'query_rs', 'RuleApplyLoan_cashon_num']]
br_df_cae.loc[:, f'{product_list[0].lower()}'] = br_df_cae.swifter.apply(lambda x: jsonpath(json.loads(x.query_rs), f'$..hit_rules.{product_list[0].lower()}') if product_list[0].lower() in x.query_rs else np.nan, axis=1)

# 获取申请意向-3C租赁（产品代号：Rule_D_ApplyLoanC）
br_df_alc = br_df[br_df.Rule_D_ApplyLoanC_num==1][['trace_id', 'query_rs', 'Rule_D_ApplyLoanC_num']]
br_df_alc.loc[:, f'{product_list[1].lower()}'] = br_df_alc.swifter.apply(lambda x: jsonpath(json.loads(x.query_rs), f'$..hit_rules.{product_list[1].lower()}') if product_list[1].lower() in x.query_rs else np.nan, axis=1)

# 获取信贷风险勘测（产品代号：Rule_D_ApplyLoanUsury）
br_df_fcj = br_df[br_df.Rule_D_ApplyLoanUsury_num==1][['trace_id', 'query_rs', 'Rule_D_ApplyLoanUsury_num']]
try:
    br_df_fcj.loc[:, f'{product_list[2].lower()}'] = br_df_fcj.swifter.apply(lambda x: jsonpath(json.loads(x.query_rs), f'$..hit_rules.{product_list[2].lower()}') if product_list[2].lower() in x.query_rs else np.nan, axis=1)
except:
    br_df_fcj.loc[:, f'{product_list[2].lower()}'] = []

# 获取信贷意向验证当日版（产品代号：ApplyLoan_d）
br_df_ald = br_df[br_df.ApplyLoan_d_num==1][['trace_id', 'query_rs', 'ApplyLoan_d_num']]
br_df_ald.loc[:, f'{product_list[3].lower()}'] = br_df_ald.swifter.apply(lambda x: jsonpath(json.loads(x.query_rs), f'$..{product_list[3]}') if product_list[3].lower() in x.query_rs else np.nan, axis=1)

# 获取信贷意向验证月度版（产品代号：ApplyLoanMon）
br_df_alm = br_df[br_df.ApplyLoanMon_num==1][['trace_id', 'query_rs', 'ApplyLoanMon_num']]
br_df_alm.loc[:, f'{product_list[4].lower()}'] = br_df_alm.swifter.apply(lambda x: jsonpath(json.loads(x.query_rs), f'$..{product_list[4]}') if product_list[4].lower() in x.query_rs else np.nan, axis=1)

# 获取信贷意向验证（产品代号：ApplyLoanStr）
br_df_als = br_df[br_df.ApplyLoanStr_num==1][['trace_id', 'query_rs', 'ApplyLoanStr_num']]
br_df_als.loc[:, f'{product_list[5].lower()}'] = br_df_als.swifter.apply(lambda x: jsonpath(json.loads(x.query_rs), f'$..{product_list[5]}') if product_list[5].lower() in x.query_rs else np.nan, axis=1)

# 获取申请意向-3C租赁（产品代号：ApplyLoanC）
br_df_alc2 = br_df[br_df.ApplyLoanC_num==1][['trace_id', 'query_rs', 'ApplyLoanC_num']]
br_df_alc2.loc[:, f'{product_list[6].lower()}'] = br_df_alc2.swifter.apply(lambda x: jsonpath(json.loads(x.query_rs), f'$..{product_list[6]}') if product_list[6].lower() in x.query_rs else np.nan, axis=1)

# 获取信贷风险勘测（产品代号：ApplyLoanUsury）
br_df_alu = br_df[br_df.ApplyLoanUsury_num==1][['trace_id', 'query_rs', 'ApplyLoanUsury_num']]
try:
    br_df_alu.loc[:, f'{product_list[7].lower()}'] = br_df_alu.swifter.apply(lambda x: jsonpath(json.loads(x.query_rs), f'$..{product_list[7]}') if product_list[7].lower() in x.query_rs else np.nan, axis=1)
except:
    br_df_alu.loc[:, f'{product_list[7].lower()}'] = []

Dask Apply:   0%|          | 0/25 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/15419 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/15419 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/15419 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/15419 [00:00<?, ?it/s]

...

Pandas Apply:   0%|          | 0/6 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/25 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/25 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/25 [00:00<?, ?it/s]

Dask Apply:   0%|          | 0/25 [00:00<?, ?it/s]

# 产品数据获取

In [10]:
def traverse_dict_generator(d, parent_key='', sep='_'):
    """
    递归遍历嵌套字典的生成器版本。
    参数:
        d (dict): 要遍历的字典。
        parent_key (str): 父级键名，默认为空字符串。
        sep (str): 分隔符，默认为点号'.'，用于连接多层键。
    产出:
        tuple: 包含当前层级键和值的元组。
    """
    for key, value in d.items():
        new_key = f"Rule_{key.replace('_rule', '')}{sep}{parent_key}" if parent_key else key
        if isinstance(value, dict):
            yield from traverse_dict_generator(value, new_key, sep=sep)
        else:
            yield [new_key, value]

def traverse_dict_generator_base(d, parent_key='', sep='_'):
    """
    递归遍历嵌套字典的生成器版本。
    参数:
        d (dict): 要遍历的字典。
        parent_key (str): 父级键名，默认为空字符串。
        sep (str): 分隔符，默认为点号'.'，用于连接多层键。
    产出:
        tuple: 包含当前层级键和值的元组。
    """
    for key, value in d.items():
        new_key = f"{parent_key}{sep}{key}" if parent_key else key
        if isinstance(value, dict):
            yield from traverse_dict_generator_base(value, new_key, sep=sep)
        else:
            yield [new_key, value]

def get_weight_name(df):
    # 获取各个变量和权重以及name
    df_new = df.copy()
    try:
        df_new.loc[:, 'Rule_Weight_key'] = df_new.swifter.apply(lambda x: [key for key, value in traverse_dict_generator(x[-1][0])][::2], axis=1)
        df_new.loc[:, 'Rule_Weight_value'] = df_new.swifter.apply(lambda x: [value for key, value in traverse_dict_generator(x[-2][0])][::2], axis=1)
        df_new.loc[:, 'Rule_Name_key'] = df_new.swifter.apply(lambda x: [key for key, value in traverse_dict_generator(x[-3][0])][1::2], axis=1)
        df_new.loc[:, 'Rule_Name_value'] = df_new.swifter.apply(lambda x: [value for key, value in traverse_dict_generator(x[-4][0])][1::2], axis=1)
    except:
        df_new.loc[:, 'Rule_Weight_key'] = []
        df_new.loc[:, 'Rule_Weight_value'] = []
        df_new.loc[:, 'Rule_Name_key'] = []
        df_new.loc[:, 'Rule_Name_value'] = []
    # 把权重和变量名展开
    df_new = df_new.explode(['Rule_Weight_key', 'Rule_Weight_value', 'Rule_Name_key', 'Rule_Name_value'])
    # 制作透视表
    df_weight_pivot = pd.pivot_table(df_new, values='Rule_Weight_value', index='trace_id', columns='Rule_Weight_key', aggfunc='first')
    df_name_pivot = pd.pivot_table(df_new, values='Rule_Name_value', index='trace_id', columns='Rule_Name_key', aggfunc='first')
    return df_weight_pivot, df_name_pivot

In [14]:
# 获取已写入的百融产品数据，随后和新数据进行拼接在重新写入
df_sheet = pd.ExcelFile('F:/需求/总需求/百融产品.xlsx')
df_cae = df_sheet.parse('RuleApplyLoan_cashon')
df_alc = df_sheet.parse('Rule_D_ApplyLoanC')
df_fcj = df_sheet.parse('Rule_D_ApplyLoanUsury')
df_ald = df_sheet.parse('ApplyLoan_d')
df_alm = df_sheet.parse('ApplyLoanMon')
df_als = df_sheet.parse('ApplyLoanStr')
df_alc2 = df_sheet.parse('ApplyLoanC')
df_alu = df_sheet.parse('ApplyLoanUsury')

In [11]:
df_weight_cae_pivot, df_name_cae_pivot = get_weight_name(br_df_cae)

Pandas Apply:   0%|          | 0/14516 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/14516 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/14516 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/14516 [00:00<?, ?it/s]

In [12]:
df_name_cae_pivot

Rule_Name_key,Rule_name_CAE005,Rule_name_CAE006,Rule_name_CAE007,Rule_name_CAE008,Rule_name_CAE009,Rule_name_CAE010,Rule_name_CAE011,Rule_name_CAE012,Rule_name_CAE013,Rule_name_CAE014,Rule_name_CAE015,Rule_name_CAE016,Rule_name_CAE017,Rule_name_CAE018,Rule_name_CAE019,Rule_name_CAE020,Rule_name_CAE021,Rule_name_CAE022,Rule_name_CAE023,Rule_name_CAE024,Rule_name_CAE025,Rule_name_CAE026,Rule_name_CAE027,Rule_name_CAE028,Rule_name_CAE029,Rule_name_CAE030,Rule_name_CAE031,Rule_name_CAE032,Rule_name_CAE033,Rule_name_CAE034,Rule_name_CAE035,Rule_name_CAE036,Rule_name_CAE037,Rule_name_CAE038,Rule_name_CAE039,Rule_name_CAE040,Rule_name_CAE041,Rule_name_CAE042,Rule_name_CAE043,Rule_name_CAE044,Rule_name_CAE045,Rule_name_CAE046,Rule_name_CAE047,Rule_name_CAE048,Rule_name_CAE049,Rule_name_CAE050,Rule_name_CAE051,Rule_name_CAE052,Rule_name_CAE053,Rule_name_CAE054,Rule_name_CAE055,Rule_name_CAE056,Rule_name_CAE057,Rule_name_CAE058,Rule_name_CAE059,Rule_name_CAE060,Rule_name_CAE061,Rule_name_CAE062,Rule_name_CAE063,Rule_name_CAE064,Rule_name_CAE065,Rule_name_CAE066,Rule_name_CAE067,Rule_name_CAE068,Rule_name_CAE069,Rule_name_CAE070,Rule_name_CAE071,Rule_name_CAE072,Rule_name_CAE073,Rule_name_CAE074,Rule_name_CAE075,Rule_name_CAE076,Rule_name_CAE077,Rule_name_CAE078,Rule_name_CAE079,Rule_name_CAE080,Rule_name_CAE081,Rule_name_CAE082,Rule_name_CAE083,Rule_name_CAE084,Rule_name_CAE085,Rule_name_CAE086,Rule_name_CAE087,Rule_name_CAE088,Rule_name_CAE089,Rule_name_CAE090,Rule_name_CAE091,Rule_name_CAE092,Rule_name_CAE093,Rule_name_CAE094,Rule_name_CAE095,Rule_name_CAE096,Rule_name_CAE097,Rule_name_CAE098,Rule_name_CAE099,Rule_name_CAE100,Rule_name_CAE101,Rule_name_CAE102,Rule_name_CAE103,Rule_name_CAE104
trace_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00023a11842b51641f8889025e23423a,NaN,NaN,NaN,NaN,NaN,近期在非银机构申请极为频繁,NaN,NaN,NaN,NaN,NaN,NaN,NaN,申请线上消费分期极为频繁,NaN,NaN,NaN,NaN,长期周末在非银机构申请极为频繁,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,近期在非银机构申请较为频繁,NaN,NaN,NaN,NaN,申请线上现金分期较为频繁,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,长期申请机构类型数较多,NaN,NaN,NaN,在网络小贷机构申请较少,在消费金融机构申请较少,某月在非银机构申请较少,NaN,近期在非银机构申请较少,长期在非银机构申请较少,近期申请线上小额现金贷较少,长期申请线上小额现金贷较少,NaN,NaN,NaN,NaN,NaN,夜间在非银机构申请较少,近期周末在非银机构申请较少,NaN,NaN,NaN,长期某月申请较少,NaN,近期申请机构类型数较少,NaN,近期申请产品类型数较少,NaN,在小贷机构申请极少,在网络小贷机构申请极少,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,近期某月申请极少,NaN,NaN,NaN,NaN,近期申请产品类型数极少,长期申请产品类型数极少
0004b68e81453b1782bd59bbb9fee975,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,在网络小贷机构申请较少,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,夜间在非银机构申请较少,NaN,长期周末在非银机构申请较少,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,在消费金融机构申请极少,申请当日在非银机构申请极少,NaN,NaN,长期在非银机构申请极少,NaN,长期申请线上小额现金贷极少,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,长期某月申请极少,长期申请极少,NaN,长期申请机构类型数极少,NaN,NaN
0008c164b723f3b29c6c0981d19deee4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,长期在非银机构申请较少,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,长期申请较少,近期申请机构类型数较少,NaN,近期申请产品类型数较少,NaN,NaN,在网络小贷机构申请极少,在消费金融机构申请极少,NaN,NaN,近期在非银机构申请极少,NaN,近期申请线上小额现金贷极少,NaN,NaN,申请线上现金分期极少,NaN,NaN,NaN,NaN,NaN,NaN,长期某月申请极少,NaN,NaN,长期申请机构类型数极少,近期申请产品类型数极少,NaN
000d75284f3f84e035a4080aae13d62b,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,在消费金融机构申请极少,NaN,某月在非银机构申请极少,NaN,NaN,近期申请线上小额现金贷极少,长期申请线上小额现金贷极少,NaN,NaN,NaN,夜间在非银机构申请极少,NaN,长期周末在非银机构申请极少,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001005ff4ba30bfaa5412c84af5b591a

In [13]:
# 信贷意向验证（产品代号：RuleApplyLoan_cashon）
df_weight_cae_pivot, df_name_cae_pivot = get_weight_name(br_df_cae)
df_weight_cae_pivot_new = pd.concat([df_cae, df_weight_cae_pivot])

# 申请意向-3C租赁（产品代号：Rule_D_ApplyLoanC）

df_weight_alc_pivot, df_name_alc_pivot = get_weight_name(br_df_alc)
df_weight_alc_pivot_new = pd.concat([df_alc, df_weight_alc_pivot])

# 信贷风险勘测（产品代号：Rule_D_ApplyLoanUsury）
df_weight_fcj_pivot, df_name_fcj_pivot = get_weight_name(br_df_fcj)
df_weight_fcj_pivot_new = pd.concat([df_fcj, df_weight_fcj_pivot])


Pandas Apply:   0%|          | 0/14516 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/14516 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/14516 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/14516 [00:00<?, ?it/s]

NameError: name 'df_cae' is not defined

# 基础数据

In [12]:
def get_key_value(df, base_key):
    df_new = df.copy()
    # 获取各个层级的字段和值
    try:
        df_new.loc[:, f'{base_key}_key'] = df_new.swifter.apply(lambda x: [f'{base_key}_'+key for key, value in traverse_dict_generator_base(x[-1][0])], axis=1)
        df_new.loc[:, f'{base_key}_value'] = df_new.swifter.apply(lambda x: [value for key, value in traverse_dict_generator_base(x[-2][0])], axis=1)
    except:
        df_new.loc[:, f'{base_key}_key'] = []
        df_new.loc[:, f'{base_key}_value'] = []
    # 将列表的值展开
    df_new = df_new.explode([f'{base_key}_key', f'{base_key}_value'])
    # 制作透视表
    df_new_pivot = pd.pivot_table(df_new, values=f'{base_key}_value', index='trace_id', columns=f'{base_key}_key', aggfunc='first')
    return df_new_pivot

In [13]:
# 信贷意向验证当日版（产品代号：ApplyLoan_d） 
df_new_ald_pivot = get_key_value(br_df_ald, 'ald')
df_new_ald_pivot_new = pd.concat([df_ald, df_new_ald_pivot])

# 信贷意向验证月度版（产品代号：ApplyLoanMon）
df_new_alm_pivot = get_key_value(br_df_alm, 'alm')
df_new_alm_pivot_new = pd.concat([df_alm, df_new_alm_pivot])

# 信贷意向验证（产品代号：ApplyLoanStr）
df_new_als_pivot = get_key_value(br_df_als, 'als')
df_new_als_pivot_new = pd.concat([df_als, df_new_als_pivot])

# 申请意向-3C租赁（产品代号：ApplyLoanC）
df_new_alc2_pivot = get_key_value(br_df_alc2, 'alc')
df_new_alc2_pivot_new = pd.concat([df_alc2, df_new_alc2_pivot])

# 信贷风险勘测（产品代号：ApplyLoanUsury）
df_new_alu_pivot = get_key_value(br_df_alu, 'alu')
df_new_alu_pivot_new = pd.concat([df_alu, df_new_alu_pivot])


Pandas Apply:   0%|          | 0/2646 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/2646 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/15133 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/15133 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/15137 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/15137 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/11576 [00:00<?, ?it/s]

Pandas Apply:   0%|          | 0/11576 [00:00<?, ?it/s]

In [14]:
with pd.ExcelWriter('F:/需求/总需求/百融产品.xlsx', engine='xlsxwriter') as writer:
    df_weight_cae_pivot_new.to_excel(writer, sheet_name='RuleApplyLoan_cashon', index=False)
    df_weight_alc_pivot_new.to_excel(writer, sheet_name='Rule_D_ApplyLoanC', index=False)
    df_weight_fcj_pivot_new.to_excel(writer, sheet_name='Rule_D_ApplyLoanUsury', index=False)
    df_new_ald_pivot_new.to_excel(writer, sheet_name='ApplyLoan_d', index=False)
    df_new_alm_pivot_new.to_excel(writer, sheet_name='ApplyLoanMon', index=False)
    df_new_als_pivot_new.to_excel(writer, sheet_name='ApplyLoanStr', index=False)
    df_new_alc2_pivot_new.to_excel(writer, sheet_name='ApplyLoanC', index=False)
    df_new_alu_pivot_new.to_excel(writer, sheet_name='ApplyLoanUsury', index=False)